
# torch.autograd를 사용한 자동 미분

신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파이다.

이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 gradient에 따라 조정된다.

이러한 변화도를 계싼하기 위해 파이토치에서는 자동 미분 엔진인 torch.autograd가 내장되어 있다.

이는 모든 계산 그래프에 대한 변화도의 자동 계산을 지원한다.

입력 x, 매개변수 w와 b, 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정할 것이다.

파이토치에서는 다음과 같이 정의할 수 있다.

![graph](img/img.png)

이 신경망에서, w와 b는 최적화를 해야 하는 매개변수이다.

따라서 이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야 한다.

이를 위해 해당 텐서에 required_grad 속성을 설정한다.

-> required_grad=True를 설정하지 못한 경우에 나중에 x.required_grad(True)로 설정 가능

In [4]:
import torch
x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected output

# parameters
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)

z = torch.matmul(x, w) + b

loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 
사실 Function 클래스의 객체이다.

이 객체는 forward 방향으로 함수를 계산하는 방법과, backward 단계에서 도함수(derivative)를 계산하는 방법을 알고 있다.

역전파 함수에 대한 참조(reference)는 텐서의 grad_fn 속성에 저장된다.


In [5]:
print('Gradient funtion for z')
print(z.grad_fn)
print('Gradient function for loss')
print(loss.grad_fn)

Gradient funtion for z
Gradient function for loss


# Gradient 계산하기

신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수를 계산해야 함

즉, x와 y의 일부 고정값에서 ∂loss/∂w와 ∂loss/∂b가 필요하다.

이러한 도함수를 계산하기 위해, loss.backward()를 호출한 다음 
w.grad와 b.grad에서 값을 가져온다.

* 연산 그래프의 leaf(잎) 노드들 중 required_grad=True로 설정된 노드들의 grad 속성만 구할 수 있음.
* 성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있음.
* 만약 동일한 그래프에서 여러번의 backward 호출이 필요하면, backward 호출 시 retain_graph=True를 전달해야 함


In [6]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0056, 0.3067, 0.2813],
        [0.0056, 0.3067, 0.2813],
        [0.0056, 0.3067, 0.2813],
        [0.0056, 0.3067, 0.2813],
        [0.0056, 0.3067, 0.2813]])
tensor([0.0056, 0.3067, 0.2813])


# 변화도 추적 멈추기
기본적으로, required_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원한다.

그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 
순전파 연산만 필요한 경우에는,

이러한 추적이나 지원이 필요 없을 수 있다.

연산 코드를 torch.no_grad() 블록으로 둘러 싸서 연산 추적을 멈출 수 있다.


In [7]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


* 또 다른 방법으로 자동 미분 멈추기

In [10]:
z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


### 변화도 추적을 멈춰야 하는 이유
1. 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시

-> 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오

 2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문이다.

-> 순전파 단계만 수행할 때 연산 속도가 향상된다.
 

# 연산 그래프에 대한 추가 정보
개념적으로, autograd는 데이터(텐서)의 실행된 모든 연산들의 기록을 Function 객체로 구성된 방향성 비순환 그래프(DAG; Directed Acyclic Graph)에 저장한다.

이 방향성 비순환 그래프의 leave(잎)은 텐서이고, root는 결과 텐서이다.

이 그래프를 뿌리에서부터 잎까지 추적하면 연쇄 법칙에 따라 변화도를 자동으로 계산할 수 있다.

### 순전파 단계에서 autograd가 동시에 수행하는 것
- 요청된 연산을 수행하여 결과 텐서 계산
- DAG에 연산의 변화도 기능(gradient function)를 유지(maintain)


### 역전파 단계에서 autograd가 수행하는 것 
#### (DAG 뿌리에서 .backward()가 호출될 때 시작)
- 각 .gard_fn으로부터 변화도 계산
- 각 텐서의 .grad 속성에 계산 결과를 accumulate
- 연쇄 법칙을 사용하여, 모든 leaf 텐서까지 전파(propagate)



파이토치에서 DAG들은 동적(dynamic)이다.

주목해야 할 중요한 점은 그래프가 처음부터(from scratch) 다시 생성된다는 것

매번 .backward()가 호출되고 나면, autograd는 새로운 그래프를 populate(채우기)하기 시작한다.

그 덕분에 모델에서 흐름 제어(control flow) 구문들을 사용할 수 있게 되는 것이다.

매번 iteration 할 때 마다 필요하면 shape나 size, operation을 바꿀 수 있다.



# Optional Reading 
# : 텐서 변화도와 Jacobian Product 계산
대부분의 경우, 스칼라 손실 함수를 가지고 일부 매개변수와 관련한 변화도를 계산해야 한다.

그러나 출력 함수가 임의의 텐서인 경우가 있다.

이럴 때, 파이토치는 실제 변화도가 아닌 야코비안 곱을 계산한다.


x : 
![v1](img/v1.png)

y : 
![v2](img/v2.png)

벡터 함수 y = f(x)에서 x에 대한 y의 변화도는 야코비안 행렬로 주어진다.

![j](img/j.png)

야코비안 행렬 자체를 계산하는 대신, 파이토치는 주어진 입력 벡터 v에 대한 야코비안 곱![j-product](jp.png)을 계산한다.


이 과정은 v를 인자로 backward를 호출하면 이뤄진다.

v의 크기는 product를 계산하려는 원래 텐서의 크기와 같아야 한다.




In [14]:
inp = torch.eye(5, requires_grad=True) # 주 성분 1인 대각 행렬
print(inp)
out = (inp+1).pow(2) # +=1 ^2
print(out)

out.backward(torch.ones_like(inp), retain_graph=True)
print('\nFirst call')
print(inp.grad)

out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call")
print(inp.grad)

inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients")
print(inp.grad)


tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)
tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


동일한 인자로 backward를 두 차례 호출하면 변화 값도 달라진다.

이는 역전파를 수행할 때, 파이토치가 gradient를 누적(accumulate)하기 때문이다.

즉, 계산된 변화도의 값이 연산 그래프 모든 노드의 grad 속성에 추가된다.

따라서 제대로 된 gradient를 계산하기 위해서는 grad 속성을 먼저 0으로 만들어야 한다.


실제 학습 과정에서는 optimizer가 이 과정을 도와준다.

- 이전에는 매개변수 없이 backward() 함수를 호출했다.
이는 본질적으로 backward(torch.tensor(1.0))을 호출하는 것과 동일하며, 신경망 훈련 중의 손실과 같은 스칼라-값 함수의 gradient를 계산하는 유용한 방법이다.